# Verify S3_BUCKET Bucket Creation

In [ ]:
import boto3
import time

session = boto3.session.Session()
region = session.region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [ ]:
setup_s3_bucket_passed = False
%store -r airflow_bucket_name
%store -r s3_mwaa_private_path
%store -r s3_mwaa_dags_private_path
%store -r airflow_env_name
%store -r airflow_vpc_name
%store -r team_role_arn
%store -r airflow_sg_id
%store -r airflow_subnet_ids

In [ ]:
!aws s3 ls $s3_mwaa_private_path

In [ ]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=airflow_bucket_name)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(airflow_bucket_name, response, e))

In [ ]:
# %store setup_s3_bucket_passed

# Create Managed Apache Airflow Environment

In [ ]:
mwaa = boto3.client("mwaa")

s3_mwaa_bucket_arn = "arn:aws:s3:::{}".format(airflow_bucket_name)

airflow_env_arn = mwaa.create_environment(
    DagS3Path="dags",
    ExecutionRoleArn=team_role_arn,
    AirflowVersion="1.10.12",
    WebserverAccessMode="PUBLIC_ONLY",
    LoggingConfiguration={
        "DagProcessingLogs": {"Enabled": True, "LogLevel": "ERROR"},
        "SchedulerLogs": {"Enabled": True, "LogLevel": "ERROR"},
        "TaskLogs": {"Enabled": True, "LogLevel": "INFO"},
        "WebserverLogs": {"Enabled": True, "LogLevel": "ERROR"},
        "WorkerLogs": {"Enabled": True, "LogLevel": "ERROR"},
    },
    MaxWorkers=3,
    Name=airflow_env_name,
    NetworkConfiguration={
        "SecurityGroupIds": [
            airflow_sg_id,
        ],
        "SubnetIds": airflow_subnet_ids,
    },
    RequirementsS3ObjectVersion="latest",
    RequirementsS3Path="requirements.txt",
    SourceBucketArn=s3_mwaa_bucket_arn,
    EnvironmentClass="mw1.small",
)

%store airflow_env_arn

# Please be patient this can take around 15 Minutes.

In [ ]:
def get_airflow_check():
    response = mwaa.get_environment(Name=airflow_env_name)
    mwaa_status = response["Environment"]["Status"]
    return mwaa_status


mwaa_status = "CREATING"

print("Checking to see if MWAA Env: {} is ready.".format(airflow_env_name))

while get_airflow_check() != "AVAILABLE":
    mwaa_status
    time.sleep(60)
    print("Still waiting for MWAA Environment...")

print("Sucess! MWAA Env: {} is ready!".format(airflow_env_name))

# PLEASE MAKE SURE THAT THE ABOVE COMMAND RAN SUCESSFULLY BEFORE CONTINUING

In [ ]:
response = mwaa.create_web_login_token(
    Name=airflow_env_name
)

webServerHostName = response["WebServerHostname"]
webToken = response["WebToken"]
airflowUIUrl = 'https://'+webServerHostName+'/aws_mwaa/aws-console-sso?login=true#'+webToken
print("Here is your AirflowUI Url:"\n)
airflowUIUrl

# Release Resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}